In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import seaborn as sb 
# 온라인 표시를 위해 그림 크기 조정
plt.rcParams['figure.figsize'] = (12.0, 10.0)

# 데이터셋 읽어오기

In [2]:
############################## read.csv ##################################
## 속성값
## : parse_dates[index or 'column'] : date형태의 칼럼을 넣는다
## : nrows = n : n개의 행만 read
## : dtype = 변수 : date type 지정

types = {'StateHoliday': np.dtype(str)}

train = pd.read_csv(r"./train.csv", parse_dates=[2], nrows=66901, dtype=types)
store = pd.read_csv(r"./store.csv")

In [ ]:
train.head()

## 데이터셋 확인

In [4]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
train.shape

(66901, 9)

In [6]:
store.shape

(1115, 10)

In [7]:
############################## 결측값 제거 ##################################

# train = train.dropna(axis=0)
# store = store.dropna(axis=0)

## 데이터셋 2개를 Store를 기준으로 merge

In [8]:
df = pd.merge(train,store, on='Store', how='left') # default값 how='left'

In [9]:
df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66896,1112,2,2015-06-02,12605,943,1,1,0,0,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
66897,1113,2,2015-06-02,7777,742,1,1,0,0,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
66898,1114,2,2015-06-02,25009,3419,1,1,0,0,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN
66899,1115,2,2015-06-02,9232,558,1,1,0,0,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"


# 결측값(=NaN) 0으로 변환

In [10]:
## NaN값 0으로 replace
df = df.fillna(0)

In [11]:
df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66896,1112,2,2015-06-02,12605,943,1,1,0,0,c,c,1880.0,4.0,2006.0,0,0.0,0.0,0
66897,1113,2,2015-06-02,7777,742,1,1,0,0,a,c,9260.0,0.0,0.0,0,0.0,0.0,0
66898,1114,2,2015-06-02,25009,3419,1,1,0,0,a,c,870.0,0.0,0.0,0,0.0,0.0,0
66899,1115,2,2015-06-02,9232,558,1,1,0,0,d,c,5350.0,0.0,0.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec"


# Date값 Year / Month / Day로 나누기

In [12]:
############# Datetime -> String 형변환 -> 구분자로 나누기 작업 #################
df['Year'] = df.Date.dt.year
df['Month'] = df.Date.dt.month
df['Day'] = df.Date.dt.day

In [13]:
#################### Date 삭제 ##########################

df.drop(labels='Date',axis=1, inplace=True)

In [14]:
df

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day
0,1,5,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,2015,7,31
1,2,5,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,7,31
2,3,5,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,7,31
3,4,5,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,0.0,0,2015,7,31
4,5,5,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0,2015,7,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66896,1112,2,12605,943,1,1,0,0,c,c,1880.0,4.0,2006.0,0,0.0,0.0,0,2015,6,2
66897,1113,2,7777,742,1,1,0,0,a,c,9260.0,0.0,0.0,0,0.0,0.0,0,2015,6,2
66898,1114,2,25009,3419,1,1,0,0,a,c,870.0,0.0,0.0,0,0.0,0.0,0,2015,6,2
66899,1115,2,9232,558,1,1,0,0,d,c,5350.0,0.0,0.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,6,2


# 인코딩 작업

In [32]:
############# 인코딩 작업을 위한 유닛값 찾아보기 ################

df['Promo2SinceYear'].unique()

array([   0., 2010., 2011., 2012., 2009., 2014., 2015., 2013.])

In [25]:
############################ 인코딩 함수 만들기 ############################

def label_encode(df, col):
    df_labelled_encode = df.copy()
    # 다른 클래스에 대해 다른 숫자를 지정하는 딕셔너리
    label_encode = {col: {'a':0, 'b':1, 'c':2, 'd':3}}
    # .replace를 사용하여 서로 다른 클래스를 숫자로 변경
    df_labelled_encode.replace(label_encode,inplace=True)
    return df_labelled_encode

In [17]:
############################ 인코딩 작업 ############################

df = label_encode(df, "Assortment")

In [21]:
############################ 인코딩 작업 ############################

df = label_encode(df, "StateHoliday")

In [26]:
############################ 인코딩 작업 ############################

df = label_encode(df, "StoreType")

In [33]:
df

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Year,Month,Day
0,1,5,5263,555,1,1,0,1,2,0,1270.0,9.0,2008.0,0,0.0,0.0,0,2015,7,31
1,2,5,6064,625,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2015,7,31
2,3,5,8314,821,1,1,0,1,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2015,7,31
3,4,5,13995,1498,1,1,0,1,2,2,620.0,9.0,2009.0,0,0.0,0.0,0,2015,7,31
4,5,5,4822,559,1,1,0,1,0,0,29910.0,4.0,2015.0,0,0.0,0.0,0,2015,7,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66896,1112,2,12605,943,1,1,0,0,2,2,1880.0,4.0,2006.0,0,0.0,0.0,0,2015,6,2
66897,1113,2,7777,742,1,1,0,0,0,2,9260.0,0.0,0.0,0,0.0,0.0,0,2015,6,2
66898,1114,2,25009,3419,1,1,0,0,0,2,870.0,0.0,0.0,0,0.0,0.0,0,2015,6,2
66899,1115,2,9232,558,1,1,0,0,3,2,5350.0,0.0,0.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2015,6,2


### 영향없는 PromoInterval 제거

In [34]:
############################ 영향없는 컬럼 drop ############################

df.drop(labels='PromoInterval',axis=1, inplace=True)
df

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day
0,1,5,5263,555,1,1,0,1,2,0,1270.0,9.0,2008.0,0,0.0,0.0,2015,7,31
1,2,5,6064,625,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,2015,7,31
2,3,5,8314,821,1,1,0,1,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,2015,7,31
3,4,5,13995,1498,1,1,0,1,2,2,620.0,9.0,2009.0,0,0.0,0.0,2015,7,31
4,5,5,4822,559,1,1,0,1,0,0,29910.0,4.0,2015.0,0,0.0,0.0,2015,7,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66896,1112,2,12605,943,1,1,0,0,2,2,1880.0,4.0,2006.0,0,0.0,0.0,2015,6,2
66897,1113,2,7777,742,1,1,0,0,0,2,9260.0,0.0,0.0,0,0.0,0.0,2015,6,2
66898,1114,2,25009,3419,1,1,0,0,0,2,870.0,0.0,0.0,0,0.0,0.0,2015,6,2
66899,1115,2,9232,558,1,1,0,0,3,2,5350.0,0.0,0.0,1,22.0,2012.0,2015,6,2


---

# Datetime -> String 형변환(사용X)

###  나중에 df_test['Year'] = df_test['Date'].dt.year 사용해서 변경함

In [155]:
df_data = df['Date']
type(df_data[0])

numpy.int64

In [27]:
ts_str = []
for i in range(len(df_data)):
    ts_str.append(str(df_data[i]))

In [28]:
ts_str

['2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:00:00',
 '2015-07-31 00:

In [29]:
#train01
X = []
date_data = []
time_data = []

for i in range(len(ts_str)):
    X = ts_str[i].split()
    #print(X)
    date_data.append(X[0])
    time_data.append(X[1])

In [35]:
##############time_data는 전부 다 00:00:00이므로 필요 없음##############

set(time_data)

{'00:00:00'}

In [36]:
import re

result = []
for i in range(len(date_data)):
    X = re.sub("-", "", date_data[i])
    result.append(X)

result

['20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',
 '20150731',

---

---

# target / featrues 지정

In [45]:
######################### target 값 지정 #############################
df_target_Sales = df['Sales']
df_target_Customers = df['Customers']

In [46]:
######################### feature 값 지정 #############################

df_features = df.copy() # copy

######################### target 값 삭제 #############################
df_features.drop(labels='Sales',axis=1, inplace=True)
df_features.drop(labels='Customers',axis=1, inplace=True)

In [47]:
df_features.head()

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day
0,1,5,1,1,0,1,2,0,1270.0,9.0,2008.0,0,0.0,0.0,2015,7,31
1,2,5,1,1,0,1,0,0,570.0,11.0,2007.0,1,13.0,2010.0,2015,7,31
2,3,5,1,1,0,1,0,0,14130.0,12.0,2006.0,1,14.0,2011.0,2015,7,31
3,4,5,1,1,0,1,2,2,620.0,9.0,2009.0,0,0.0,0.0,2015,7,31
4,5,5,1,1,0,1,0,0,29910.0,4.0,2015.0,0,0.0,0.0,2015,7,31


In [48]:
df_features.shape

(66901, 17)

# 데이터셋 나누기

In [49]:
############################ train / test data 나누기(Sales) ############################
# train_features
# train_target

from sklearn.model_selection import train_test_split
X_train_S, X_test_S, y_train_S, y_test_S = train_test_split(df_features, df_target_Sales, 
                                                    test_size=0.2, 
                                                    random_state=150, # 내부적으로 사용되는 난수 값
                                                    shuffle=True) #  데이터를 분리할 때 랜덤으로 분리할지의 유무
                                                     
# stratify (array) : 분리하기 이전의 클래스 비율을 분리하고 나서도 유지하기 위해 설정해야 하는 값 (종속변수의 컬럼을 넣어주면 됩니다.)
# 원본 데이터의 클래스 비율이 8:2라면 분리된 train data, valid(test) data의 클래스 비율도 8:2가 유지됩니다.
# 따라서 stratify 파라미터는 분류 문제에서만 사용 가능합니다.

In [50]:
############################ train / test data 나누기(Customers) ############################
# train_features
# train_target

from sklearn.model_selection import train_test_split
X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(df_features, df_target_Customers, 
                                                    test_size=0.2, 
                                                    random_state=150, # 내부적으로 사용되는 난수 값
                                                    shuffle=True) #  데이터를 분리할 때 랜덤으로 분리할지의 유무
                                                     
# stratify (array) : 분리하기 이전의 클래스 비율을 분리하고 나서도 유지하기 위해 설정해야 하는 값 (종속변수의 컬럼을 넣어주면 됩니다.)
# 원본 데이터의 클래스 비율이 8:2라면 분리된 train data, valid(test) data의 클래스 비율도 8:2가 유지됩니다.
# 따라서 stratify 파라미터는 분류 문제에서만 사용 가능합니다.

# 데이터셋 확인 shape()

In [51]:
print(X_train_S.shape)
print(X_test_S.shape)
print(y_train_S.shape)
print(y_test_S.shape)

(53520, 17)
(13381, 17)
(53520,)
(13381,)


In [52]:
print(X_train_C.shape)
print(X_test_C.shape)
print(y_train_C.shape)
print(y_test_C.shape)

(53520, 17)
(13381, 17)
(53520,)
(13381,)


# DecisionTreeClassifier 모델 학습

In [53]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

regressor_S = DecisionTreeRegressor()               # 종속변수가 현재 범주형
regressor_S.fit(X_train_S, y_train_S) # feature, target

DecisionTreeRegressor()

In [54]:
regressor_C = DecisionTreeRegressor() #종속변수가 현재 범주형
# random_state=1, min_samples_leaf=35, max_depth=10
regressor_C.fit(X_train_C, y_train_C) # feature, target

DecisionTreeRegressor()

## target(=Sales) MSE 확인하기

In [55]:
from sklearn.metrics import make_scorer

# kaggle에 따른 오류 계산 함수
def rmspe(y, y_hat):
    return np.sqrt(np.mean(((y - y_hat) / y) ** 2))

rmpse_scorer = make_scorer(rmspe, greater_is_better = False) # Loss function

def score(regressor_S, X_train_S, y_train_S, y_test_S, y_hat):
    score = cross_val_score(regressor_S, X_train_S, y_train_S, scoring=rmpse_scorer, cv=5)
    print('Mean', score.mean())
    print('Variance', score.var())
    print('RMSPE', rmspe(y_test_S, y_hat))

In [56]:
y_hat = regressor_S.predict(X_test_S)
score(regressor_S, X_train_S, y_train_S, y_test_S, y_hat)

Mean -0.20059878537993253
Variance 5.7036109772558035e-05
RMSPE 0.19025687680080464


## target(=Customers) MSE 확인하기

In [57]:
from sklearn.metrics import make_scorer

# kaggle에 따른 오류 계산 함수
def rmspe(y, y_hat):
    return np.sqrt(np.mean(((y - y_hat) / y) ** 2))

rmpse_scorer = make_scorer(rmspe, greater_is_better = False) # Loss function

def score(regressor_C, X_train_C, y_train_C, y_test_C, y_hat):
    score = cross_val_score(regressor_C, X_train_C, y_train_C, scoring=rmpse_scorer, cv=5)
    print('Mean', score.mean())
    print('Variance', score.var())
    print('RMSPE', rmspe(y_test_C, y_hat))

In [58]:
y_hat = regressor_C.predict(X_test_C)
score(regressor_C, X_train_C, y_train_C, y_test_C, y_hat)

Mean -0.12560754633051668
Variance 1.4458452624197186e-05
RMSPE 0.13041607800571187


# 예측하기

In [73]:
pred = regressor_S.predict(X_test_S)

In [74]:
####################### 리스트 - > 데이터프레임 변환 #########################
df_Sales = pd.DataFrame(pred, columns = ['Sales'])
df_Sales

,Sales
0,14912.0
1,3612.0
2,8547.0
3,10678.0
4,5691.0
...,...
13376,6808.0
13377,5485.0
13378,4907.0
13379,9943.0


In [75]:
pred = regressor_C.predict(X_test_C)

In [76]:
####################### 리스트 - > 데이터프레임 변환 #########################

df_Customers = pd.DataFrame(pred, columns = ['Customers'])
df_Customers

,Customers
0,407.0
1,521.0
2,683.0
3,798.0
4,795.0
...,...
13376,667.0
13377,504.0
13378,452.0
13379,879.0


---

---

# 예측을 위한 데이터 불러오기(test.csv)

In [77]:
types = {'StateHoliday': np.dtype(str)}

df_test = pd.read_csv(r"./test.csv", parse_dates=[3], dtype=types)

In [79]:
df_test

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0
41084,41085,1112,6,2015-08-01,1.0,0,0,0
41085,41086,1113,6,2015-08-01,1.0,0,0,0
41086,41087,1114,6,2015-08-01,1.0,0,0,0


# Date 쪼개기

In [80]:
############# Datetime -> String 형변환 -> 구분자로 나누기 작업 #################
df_test['Year'] = df_test['Date'].dt.year
df_test['Month'] = df_test['Date'].dt.month
df_test['Day'] = df_test['Date'].dt.day

In [81]:
df_test

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day
0,1,1,4,2015-09-17,1.0,1,0,0,2015,9,17
1,2,3,4,2015-09-17,1.0,1,0,0,2015,9,17
2,3,7,4,2015-09-17,1.0,1,0,0,2015,9,17
3,4,8,4,2015-09-17,1.0,1,0,0,2015,9,17
4,5,9,4,2015-09-17,1.0,1,0,0,2015,9,17
...,...,...,...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0,2015,8,1
41084,41085,1112,6,2015-08-01,1.0,0,0,0,2015,8,1
41085,41086,1113,6,2015-08-01,1.0,0,0,0,2015,8,1
41086,41087,1114,6,2015-08-01,1.0,0,0,0,2015,8,1


### 의미없는 데이터 drop

In [82]:
df_test.drop(labels='Date',axis=1, inplace=True)

In [83]:
df_test.drop(labels='Id',axis=1, inplace=True)

In [84]:
df_test

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day
0,1,4,1.0,1,0,0,2015,9,17
1,3,4,1.0,1,0,0,2015,9,17
2,7,4,1.0,1,0,0,2015,9,17
3,8,4,1.0,1,0,0,2015,9,17
4,9,4,1.0,1,0,0,2015,9,17
...,...,...,...,...,...,...,...,...,...
41083,1111,6,1.0,0,0,0,2015,8,1
41084,1112,6,1.0,0,0,0,2015,8,1
41085,1113,6,1.0,0,0,0,2015,8,1
41086,1114,6,1.0,0,0,0,2015,8,1


# 인코딩 작업

In [85]:
df_test = pd.merge(df_test,store, on='Store', how='left') # default값 how='left'

In [86]:
############################ 인코딩된 결과 확인 ############################

df_test

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,4,1.0,1,0,0,2015,9,17,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,3,4,1.0,1,0,0,2015,9,17,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,7,4,1.0,1,0,0,2015,9,17,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
3,8,4,1.0,1,0,0,2015,9,17,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
4,9,4,1.0,1,0,0,2015,9,17,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41083,1111,6,1.0,0,0,0,2015,8,1,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
41084,1112,6,1.0,0,0,0,2015,8,1,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
41085,1113,6,1.0,0,0,0,2015,8,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
41086,1114,6,1.0,0,0,0,2015,8,1,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [87]:
############################ NaN값 0으로 초기화 ############################

df_test = df_test.fillna(0)

In [88]:
df_test['StateHoliday'].unique()

array(['0', 'a'], dtype=object)

In [93]:
df_test['StoreType'].unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [97]:
df_test['Assortment'].unique()

array(['a', 'c', 'b'], dtype=object)

In [98]:
############################ 인코딩 함수 만들기 ############################

def label_encode(df, col):
    df_labelled_encode = df.copy()
    # 다른 클래스에 대해 다른 숫자를 지정하는 딕셔너리
    label_encode = {col: {'a':0, 'b':1, 'c':2}}
    # .replace를 사용하여 서로 다른 클래스를 숫자로 변경
    df_labelled_encode.replace(label_encode,inplace=True)
    return df_labelled_encode

In [90]:
############################ 인코딩 작업 ############################

df_test = label_encode(df_test, "StateHoliday")

In [95]:
############################ 인코딩 작업 ############################

df_test = label_encode(df_test, "StoreType")

In [99]:
############################ 인코딩 작업 ############################

df_test = label_encode(df_test, "Assortment")

## 제대로 인코딩 됐는지 확인하는 작업

In [100]:
df_test['Assortment'].unique()

array([0, 2, 1], dtype=int64)

## 의미없는 데이터 PromoInterval 제거

In [101]:
df_test.drop(labels='PromoInterval',axis=1, inplace=True)

In [102]:
df_test

,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
0,1,4,1.0,1,0,0,2015,9,17,2,0,1270.0,9.0,2008.0,0,0.0,0.0
1,3,4,1.0,1,0,0,2015,9,17,0,0,14130.0,12.0,2006.0,1,14.0,2011.0
2,7,4,1.0,1,0,0,2015,9,17,0,2,24000.0,4.0,2013.0,0,0.0,0.0
3,8,4,1.0,1,0,0,2015,9,17,0,0,7520.0,10.0,2014.0,0,0.0,0.0
4,9,4,1.0,1,0,0,2015,9,17,0,2,2030.0,8.0,2000.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41083,1111,6,1.0,0,0,0,2015,8,1,0,0,1900.0,6.0,2014.0,1,31.0,2013.0
41084,1112,6,1.0,0,0,0,2015,8,1,2,2,1880.0,4.0,2006.0,0,0.0,0.0
41085,1113,6,1.0,0,0,0,2015,8,1,0,2,9260.0,0.0,0.0,0,0.0,0.0
41086,1114,6,1.0,0,0,0,2015,8,1,0,2,870.0,0.0,0.0,0,0.0,0.0


# 예측하기

In [103]:
####################### 리스트 - > 데이터프레임 변환 #########################
pred = regressor_S.predict(df_test)
df_Sales = pd.DataFrame(pred, columns = ['Sales'])
df_Sales

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,Sales
0,5881.0
1,6457.0
2,7961.0
3,10288.0
4,5881.0
...,...
41083,5650.0
41084,7597.0
41085,22017.0
41086,22017.0


In [104]:
####################### 리스트 - > 데이터프레임 변환 #########################
pred = regressor_C.predict(df_test)
df_Customers = pd.DataFrame(pred, columns = ['Customers'])
df_Customers

C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,Customers
0,357.0
1,466.0
2,357.0
3,357.0
4,357.0
...,...
41083,312.0
41084,287.0
41085,3784.0
41086,3784.0


# 예측한 결과 합치기(merge)

In [107]:
##################### merge하기 위해서 index값 매기기 #####################
df_Sales = df_Sales.reset_index()
df_Sales

,index,Sales
0,0,5881.0
1,1,6457.0
2,2,7961.0
3,3,10288.0
4,4,5881.0
...,...,...
41083,41083,5650.0
41084,41084,7597.0
41085,41085,22017.0
41086,41086,22017.0


In [108]:
##################### merge하기 위해서 index값 매기기 #####################

df_Customers = df_Customers.reset_index()
df_Customers

,index,Customers
0,0,357.0
1,1,466.0
2,2,357.0
3,3,357.0
4,4,357.0
...,...,...
41083,41083,312.0
41084,41084,287.0
41085,41085,3784.0
41086,41086,3784.0


In [109]:
df_submit = pd.merge(df_Sales,df_Customers, on='index', how='left') # default값 how='left'

In [110]:
df_submit

,index,Sales,Customers
0,0,5881.0,357.0
1,1,6457.0,466.0
2,2,7961.0,357.0
3,3,10288.0,357.0
4,4,5881.0,357.0
...,...,...,...
41083,41083,5650.0,312.0
41084,41084,7597.0,287.0
41085,41085,22017.0,3784.0
41086,41086,22017.0,3784.0


In [111]:
df_submit.rename(columns = {'index':'Id'}, inplace=True)

In [112]:
df_submit

,Id,Sales,Customers
0,0,5881.0,357.0
1,1,6457.0,466.0
2,2,7961.0,357.0
3,3,10288.0,357.0
4,4,5881.0,357.0
...,...,...,...
41083,41083,5650.0,312.0
41084,41084,7597.0,287.0
41085,41085,22017.0,3784.0
41086,41086,22017.0,3784.0


# 최종결과

In [113]:
sample_submission = pd.read_csv(r"./sample_submission.csv")
sample_submission

,Id,Sales
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
41083,41084,0
41084,41085,0
41085,41086,0
41086,41087,0


In [114]:
sample_submission.drop(labels='Sales',axis=1, inplace=True)

In [115]:
submit = pd.merge(sample_submission,df_submit, on='Id', how='left') # default값 how='left'

In [116]:
submit.isnull().sum()

Id           0
Sales        1
Customers    1
dtype: int64

In [117]:
submit = submit.dropna(axis=0)

In [118]:
submit

,Id,Sales,Customers
0,1,6457.0,466.0
1,2,7961.0,357.0
2,3,10288.0,357.0
3,4,5881.0,357.0
4,5,5881.0,357.0
...,...,...,...
41082,41083,5650.0,312.0
41083,41084,7597.0,287.0
41084,41085,22017.0,3784.0
41085,41086,22017.0,3784.0


# 제출

In [93]:
import os

submit.to_csv("submission.csv", index = True)